<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import numpy as np

In [15]:
X = np.array([[0, 0, 1],
                [0, 1, 1],
                [1, 0, 1],
                [0, 1, 0],
                [1, 0, 0],
                [1, 1, 1],
                [0, 0, 0]])
y = np.array([[0], [1], [1], [1], [1], [0], [0]])

In [17]:
class NeuralNetwork:
    def __init__(self, inputs, hiddenNodes, outputNodes):
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        
        self.hidden_sum = np.dot(X, self.weights1)
        
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        self.o_error = y - o
        
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        self.z2_error = self.o_delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        
        self.weights1 += X.T.dot(self.z2_delta)
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [14]:
nn = NeuralNetwork(3, 5, 1)

for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
        
    nn.train(X,y)

+---------EPOCH 1---------+
Input: 
 [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
Actual Output: 
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]


ValueError: shapes (1797,64) and (3,5) not aligned: 64 (dim 1) != 3 (dim 0)

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [2]:
from sklearn import datasets

digits = datasets.load_digits()

In [3]:
digits.data.shape

(1797, 64)

In [4]:
X = digits["data"]
y = digits['target']

In [ ]:
X = digits["data"].reshape(64)

In [5]:
X.shape

(1797, 64)

In [6]:
y.shape

(1797,)

In [27]:
X[0].shape

(64,)

In [8]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)

In [9]:
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [12]:
class NeuralNetwork:
    def __init__(self, inputs, hiddenNodes, outputNodes):
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        
        self.hidden_sum = np.dot(X, self.weights1)
        
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        self.activated_output = self.sigmoid(self.output_sum)

        return self.activated_output
    
    def backward(self, X, y, o):
        self.o_error = y - o
        
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        self.z2_error = self.o_delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        
        self.weights1 += X.T.dot(self.z2_delta)
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [15]:
nn = NeuralNetwork(64, 100, 10)

for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
        
    nn.train(X,y)

+---------EPOCH 1---------+
Input: 
 [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
Actual Output: 
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
Predicted Output: 
 [[1.01918473e-05 7.32344300e-01 1.39428853e-01 ... 9.99990605e-01
  2.03140741e-01 4.52615812e-01]
 [2.19396832e-05 9.99990502e-01 8.27863706e-04 ... 9.99999982e-01
  8.16586577e-05 2.42090363e-01]
 [2.19446384e-06 2.42853903e-02 9.99171217e-01 ... 9.99989797e-01
  2.27071704e-04 5.92589770e-04]
 ...
 [2.42607741e-06 3.90785039e-01 1.41676437e-02 ... 9.99865992e-01
  4.55102134e-04 8.90859584e-02]
 [4.21983178e-07 8.61223413e-01 8.96542297e-01 ... 9.99940659e-01
  5.70308394e-04 1.51383555e-01]
 [2.11720235e-02 7.32575966e-01 2.03260715e-01 ... 9.99987420e-01
  4.17677599

In [18]:
print("Predicted Output: \n" + str(nn.feed_forward(X))) 
print("Loss: \n" + str(np.mean(np.square(y - nn.feed_forward(X))))) # mean sum squared loss

Predicted Output: 
[[9.77772342e-034 0.00000000e+000 0.00000000e+000 ... 1.48660431e-055
  1.59274016e-170 0.00000000e+000]
 [1.40783399e-035 0.00000000e+000 0.00000000e+000 ... 3.68568988e-056
  1.65544554e-175 0.00000000e+000]
 [4.32026502e-036 0.00000000e+000 0.00000000e+000 ... 1.21374049e-058
  5.20476450e-181 0.00000000e+000]
 ...
 [4.38853414e-036 0.00000000e+000 0.00000000e+000 ... 1.18771390e-058
  5.01092664e-181 0.00000000e+000]
 [1.40436317e-032 0.00000000e+000 0.00000000e+000 ... 7.68771001e-053
  7.79324847e-165 0.00000000e+000]
 [1.13421403e-035 0.00000000e+000 0.00000000e+000 ... 3.19744499e-059
  5.03669410e-182 0.00000000e+000]]
Loss: 
0.1


## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [8]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X / 255

# digits = 10
# examples = y.shape[0]

# y = y.reshape(1, examples)

# Y_new = np.eye(digits)[y.astype('int32')]
# Y_new = Y_new.T.reshape(digits, examples)

# m = 60000
# m_test = X.shape[0] - m

# X_train, X_test = X[:m].T, X[m:].T
# Y_train, Y_test = Y_new[:, :m], Y_new[:, m:]

# shuffle_index = np.random.permutation(m)
# X_train, Y_train = X_train[:, shuffle_index], Y_train[:, shuffle_index]
# X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

In [9]:
X.shape

(70000, 784)

In [11]:
y.shape

(70000,)

In [4]:
from sklearn.model_selection import train_test_split
X = X.reshape(X.shape[0],8,8,1)
# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle= True)

In [5]:
x_train.shape

(1203, 8, 8, 1)

In [6]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

In [10]:
def cnn_mnist_model():
    model = Sequential()
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(8,8,1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
#     model.summary()
    
    model.compile(loss='categorical_crossentropy',
             metrics=['accuracy'],
             optimizer='adam')
    return model

In [20]:
# model = cnn_mnist_model()

model.fit(x_train, y_train, batch_size=20, epochs=500)

Epoch 1/500
1203/1203 [==============================] - 0s 379us/sample - loss: 0.0467 - acc: 0.9859
Epoch 2/500
1203/1203 [==============================] - 0s 380us/sample - loss: 0.0297 - acc: 0.9925
Epoch 3/500
1203/1203 [==============================] - 0s 376us/sample - loss: 0.0383 - acc: 0.9875
Epoch 4/500
1203/1203 [==============================] - 0s 380us/sample - loss: 0.0388 - acc: 0.9900
Epoch 5/500
1203/1203 [==============================] - 1s 532us/sample - loss: 0.0623 - acc: 0.9859
Epoch 6/500
1203/1203 [==============================] - 0s 383us/sample - loss: 0.0437 - acc: 0.9875
Epoch 7/500
1203/1203 [==============================] - 0s 384us/sample - loss: 0.0529 - acc: 0.9867
Epoch 8/500
1203/1203 [==============================] - 0s 392us/sample - loss: 0.0522 - acc: 0.9867
Epoch 9/500
1203/1203 [==============================] - 1s 547us/sample - loss: 0.0349 - acc: 0.9892
Epoch 10/500
1203/1203 [==============================] - 0s 396us/sample - loss: 

In [21]:
scores = model.evaluate(x_valid, y_valid)
print(f'{model.metrics_names[1]}: {scores[1]*100}')

594/594 [==============================] - 0s 69us/sample - loss: 0.0845 - acc: 0.9865
acc: 98.65319728851318
